# Polarizability using automatic differentiation

Simple example for computing properties using (forward-mode)
automatic differentiation.
For a more classical approach and more details about computing polarizabilities,
see Polarizability by linear response.

In [1]:
using DFTK
using LinearAlgebra
using ForwardDiff

# Construct PlaneWaveBasis given a particular electric field strength
# Again we take the example of a Helium atom.
function make_basis(ε::T; a=10., Ecut=30) where {T}
    lattice=T(a) * I(3)  # lattice is a cube of $a$ Bohrs
    # Helium at the center of the box
    atoms     = [ElementPsp(:He, psp=load_psp("hgh/lda/He-q2"))]
    positions = [[1/2, 1/2, 1/2]]

    model = model_DFT(lattice, atoms, positions, [:lda_x, :lda_c_vwn];
                      extra_terms=[ExternalFromReal(r -> -ε * (r[1] - a/2))],
                      symmetries=false)
    PlaneWaveBasis(model; Ecut, kgrid=[1, 1, 1])  # No k-point sampling on isolated system
end

# dipole moment of a given density (assuming the current geometry)
function dipole(basis, ρ)
    @assert isdiag(basis.model.lattice)
    a  = basis.model.lattice[1, 1]
    rr = [a * (r[1] - 1/2) for r in r_vectors(basis)]
    sum(rr .* ρ) * basis.dvol
end

# Function to compute the dipole for a given field strength
function compute_dipole(ε; tol=1e-8, kwargs...)
    scfres = self_consistent_field(make_basis(ε; kwargs...); tol)
    dipole(scfres.basis, scfres.ρ)
end;

With this in place we can compute the polarizability from finite differences
(just like in the previous example):

In [2]:
polarizability_fd = let
    ε = 0.01
    (compute_dipole(ε) - compute_dipole(0.0)) / ε
end

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -2.770597830847                   -0.53    9.0         
  2   -2.772136554620       -2.81       -1.29    1.0    155ms
  3   -2.772169763800       -4.48       -2.71    1.0    184ms
  4   -2.772170719398       -6.02       -3.79    2.0    180ms
  5   -2.772170722787       -8.47       -4.34    2.0    205ms
  6   -2.772170723011       -9.65       -5.50    1.0    164ms
  7   -2.772170723015      -11.35       -6.10    2.0    193ms
  8   -2.772170723015      -13.35       -6.67    1.0    169ms
  9   -2.772170723015      -13.83       -7.43    1.0    193ms
 10   -2.772170723015   +    -Inf       -8.13    2.0    205ms
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -2.770718026057                   -0.53    8.0         
  2   -2.772053607575       -2.87       -1.30    1.0    197ms
  3   -2.7

1.7735580977411138

We do the same thing using automatic differentiation. Under the hood this uses
custom rules to implicitly differentiate through the self-consistent
field fixed-point problem.

In [3]:
polarizability = ForwardDiff.derivative(compute_dipole, 0.0)
println()
println("Polarizability via ForwardDiff:       $polarizability")
println("Polarizability via finite difference: $polarizability_fd")

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -2.770754175823                   -0.52    9.0         
  2   -2.772059744076       -2.88       -1.32    1.0    156ms
  3   -2.772082946381       -4.63       -2.42    1.0    208ms
  4   -2.772083331350       -6.41       -3.11    1.0    159ms
  5   -2.772083417641       -7.06       -4.35    2.0    236ms
  6   -2.772083417747       -9.97       -4.58    1.0    164ms
  7   -2.772083417809      -10.21       -5.72    1.0    180ms
  8   -2.772083417811      -11.71       -6.08    2.0    190ms
  9   -2.772083417811      -13.73       -6.46    1.0    175ms
 10   -2.772083417811      -13.92       -7.38    1.0    196ms
 11   -2.772083417811      -13.72       -7.47    2.0    217ms
 12   -2.772083417811   +  -14.12       -7.87    1.0    175ms
 13   -2.772083417811      -14.12       -8.34    1.0    215ms

Polarizability via ForwardDiff:       1.7725349799623096
Polarizabilit